In [1]:
import MySQLdb

In [2]:
db = MySQLdb.connect(host="ece457project.chwjf5irbz2p.us-east-2.rds.amazonaws.com",
                    user="aiproject2018",         
                    passwd="aiproject2018",  
                    db="bitcoinproject"
                    )

In [4]:
cur=db.cursor()

In [6]:
cur.execute("use bitcoinproject;")

0L

In [226]:
cur.execute("describe coindesk_articles;")
cur.fetchall()

(('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('title', 'varchar(255)', 'YES', '', None, ''),
 ('body', 'text', 'YES', '', None, ''),
 ('date', 'datetime', 'YES', '', None, ''))

In [309]:
dt="2018-03-29"
cur.execute("SELECT * FROM coindesk_articles WHERE date = %s",(dt,))
a=cur.fetchall()

table={
    "id":[i[0] for i in a],
    "headline":[i[1] for i in a],
    "body":[i[2] for i in a],
    "date":[i[3] for i in a]
}

import pandas as pd

df = pd.DataFrame(table)
df.loc[0][2]

'Ether Drops Below $400 to Hit Lowest Price Since November'

In [323]:
for i,j in enumerate(table["body"]):
    table["body"][i]=preproc(j)
table["headline"]

['Ether Drops Below $400 to Hit Lowest Price Since November',
 "Intel Wants to Patent a Bitcoin Mining Hardware 'Accelerator'",
 'Email Service MailChimp to Block ICO, Crypto Marketing',
 "Game Giant Unity to Work With Kik's ICO Token",
 'Scammers Are Posing As Crypto Exchange Support Staff, Says FBI',
 "'Colored Coins' Startup Coinprism Is Shutting Down",
 'Major Cryptocurrencies Are Hitting 2018 Lows Today',
 'Report: Korean Insurer Denies Claim from Bankrupt Crypto Exchange',
 'UK Financial Watchdog Warns Over Unregistered Crypto Brokerage',
 "China's Central Bank Eyes Crypto as Possible Yuan Risk",
 "Bitcoin's Lightning Network Is Live, But Can It Keep from Going Corporate?",
 '$7K Next? Bitcoin Bears Pull Off Downside Break',
 'West Virginia Piloting Blockchain Voting App in Senate Election',
 "State Digital Currency Still On Agenda, Says China's Blockchain Lead",
 "What It's Like to Talk to the SEC About Your ICO"]

In [319]:
import re
def preproc(body):
    body=body.strip()
    body=re.sub(r'\s(\n){2,}','. ',body)
    body=re.sub(r'(\n)+','. ',body)
    body=re.sub(r'(\xc2\xa0)+',' ',body)
    idx=body.find('Shutterstock')
    body=body[:idx]
    idx2=body.rfind('.')
    body=body[:idx2]+"."
    return body

[datetime.datetime(2018, 3, 30, 0, 0),
 datetime.datetime(2018, 3, 30, 0, 0),
 datetime.datetime(2018, 3, 30, 0, 0),
 datetime.datetime(2018, 3, 30, 0, 0),
 datetime.datetime(2018, 3, 30, 0, 0),
 datetime.datetime(2018, 3, 30, 0, 0),
 datetime.datetime(2018, 3, 30, 0, 0),
 datetime.datetime(2018, 3, 30, 0, 0),
 datetime.datetime(2018, 3, 30, 0, 0),
 datetime.datetime(2018, 3, 30, 0, 0),
 datetime.datetime(2018, 3, 30, 0, 0),
 datetime.datetime(2018, 3, 30, 0, 0)]

In [371]:
import json
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 \
import Features, EntitiesOptions, KeywordsOptions, SentimentOptions

def analyzer(req):
    nlu = NaturalLanguageUnderstandingV1(
      username='fb46fb33-3a6b-4ab1-abd3-1d564faee2e7',
      password='X2wDsDxGoBlm',
      version='2018-03-16')

    response = nlu.analyze(
      text=req,
      features=Features(
        sentiment=SentimentOptions(
          targets=['bitcoin','stock','price','ether','crypto','currency','cryptocurrency','cryptocurrencies','ethereum','ICO','coin','coins','blockchain','trade','trades','exchange'])))
    return response

In [339]:
response

{u'language': u'en',
 u'sentiment': {u'document': {u'label': u'negative', u'score': -0.753135},
  u'targets': [{u'label': u'negative',
    u'score': -0.753135,
    u'text': u'bitcoin'}]},
 u'usage': {u'features': 1, u'text_characters': 47, u'text_units': 1}}

In [370]:
for row in df.iterrows():
    a=str(row[1]["headline"])
    print a
    print analyzer(a)

Ether Drops Below $400 to Hit Lowest Price Since November
{u'usage': {u'text_characters': 57, u'features': 1, u'text_units': 1}, u'language': u'en', u'sentiment': {u'document': {u'score': -0.721336, u'label': u'negative'}, u'targets': [{u'text': u'price', u'score': -0.721336, u'label': u'negative'}, {u'text': u'ether', u'score': -0.721336, u'label': u'negative'}]}}
Intel Wants to Patent a Bitcoin Mining Hardware 'Accelerator'
{u'usage': {u'text_characters': 61, u'features': 1, u'text_units': 1}, u'language': u'en', u'sentiment': {u'document': {u'score': 0.0, u'label': u'neutral'}, u'targets': [{u'text': u'bitcoin', u'score': 0.0, u'label': u'neutral'}]}}
Email Service MailChimp to Block ICO, Crypto Marketing
{u'usage': {u'text_characters': 54, u'features': 1, u'text_units': 1}, u'language': u'en', u'sentiment': {u'document': {u'score': -0.429803, u'label': u'negative'}, u'targets': [{u'text': u'crypto', u'score': 0.0, u'label': u'neutral'}, {u'text': u'ICO', u'score': -0.249248, u'labe

In [342]:
b

['Ether Drops Below $400 to Hit Lowest Price Since November',
 "Intel Wants to Patent a Bitcoin Mining Hardware 'Accelerator'",
 'Email Service MailChimp to Block ICO, Crypto Marketing',
 "Game Giant Unity to Work With Kik's ICO Token",
 'Scammers Are Posing As Crypto Exchange Support Staff, Says FBI',
 "'Colored Coins' Startup Coinprism Is Shutting Down",
 'Major Cryptocurrencies Are Hitting 2018 Lows Today',
 'Report: Korean Insurer Denies Claim from Bankrupt Crypto Exchange',
 'UK Financial Watchdog Warns Over Unregistered Crypto Brokerage',
 "China's Central Bank Eyes Crypto as Possible Yuan Risk",
 "Bitcoin's Lightning Network Is Live, But Can It Keep from Going Corporate?",
 '$7K Next? Bitcoin Bears Pull Off Downside Break',
 'West Virginia Piloting Blockchain Voting App in Senate Election',
 "State Digital Currency Still On Agenda, Says China's Blockchain Lead",
 "What It's Like to Talk to the SEC About Your ICO"]